<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaTableInst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaTableInst") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df=spark.read.format('csv').option('header','true').option('inferSchema',True).load('/content/drive/MyDrive/Abc/Ola_data/products.csv')

In [4]:
df.write.format('delta').mode('overwrite').saveAsTable('Products_table')

In [5]:
deltaInstance1=DeltaTable.forPath(spark,'/content/spark-warehouse/products_table')

In [6]:
deltaInstance1.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



In [7]:
spark.sql("delete from Products_table where product_id='PROD101'")

DataFrame[]

In [8]:
spark.sql('select * from Products_table').show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



In [9]:
deltaInstance1.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



both the deltainstance and table are pointing to same memory instance, hence we could see replica as same when we query using SQL and DeltaInstance

Using forName table Name

In [10]:
deltaInstance2=DeltaTable.forName(spark,'Products_table')

In [11]:
deltaInstance2.toDF().show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
|   Wireless Mouse|Electronics Hub|   PROD106| STORE01| NULL|California|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



As can be seen, using forName also pointing to same deltatable that been created above

#### Describing History

In [12]:
spark.sql('Describe History Products_table').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+--------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                             |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+---

In [13]:
deltaInstance1.history().show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+--------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                             |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+---

In [15]:
df.write.format('delta').mode('append').saveAsTable('products_table3')

In [17]:
spark.sql('describe History products_table3').show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2025-06-16 14:00:...|  null|    null|CREATE TABLE AS S...|{isManaged -> tru...|null|    null|     null|       null|  Serializable|         true|{numFiles -> 1, n...|        null|Apache-Spark/3.2....|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+-----------

In [18]:
df.createOrReplaceTempView('temp_table')


In [19]:
spark.sql('select * from temp_table').show(5)

+-----------------+---------------+----------+--------+-----+----------+-------+
|          product|          store|product_id|store_id|  MRP|     state|country|
+-----------------+---------------+----------+--------+-----+----------+-------+
|    Wired Earbuds|Electronics Hub|   PROD101| STORE01|25.99|California|    USA|
| Portable Charger|    Mobile Mart|   PROD102| STORE02| NULL|     Texas|    USA|
|Bluetooth Speaker|Electronics Hub|   PROD103| STORE01|79.50|California|    USA|
| Smart Light Bulb|   Home Gadgets|   PROD104| STORE03|15.00|   Florida|    USA|
|             NULL|    Mobile Mart|   PROD105| STORE02|49.99|     Texas|    USA|
+-----------------+---------------+----------+--------+-----+----------+-------+
only showing top 5 rows



####Delete data from Delta tables

Using deltatable location

In [20]:
spark.sql("delete from delta.`/content/spark-warehouse/products_table2` where product_id like('PROD101')")
spark.sql('select * from delta.`/content/spark-warehouse/products_table2`').show(5)

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
+-------------------+---------------+----------+--------+------+----------+-------+
only showing top 5 rows



In [21]:
spark.sql('describe history delta.`/content/spark-warehouse/products_table2`').show(truncate=False)

+-------+-----------------------+------+--------+----------------------+----------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation             |operationParameters                                                                                       |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+-------

In [22]:
deltaInstance3=DeltaTable.forPath(spark,'/content/spark-warehouse/products_table2')

In [23]:
deltaInstance3.delete('product_id like("PROD103") and country="USA"')

In [24]:
deltaInstance3.toDF().show(5)

+-------------------+---------------+----------+--------+------+----------+-------+
|            product|          store|product_id|store_id|   MRP|     state|country|
+-------------------+---------------+----------+--------+------+----------+-------+
|   Portable Charger|    Mobile Mart|   PROD102| STORE02|  NULL|     Texas|    USA|
|   Smart Light Bulb|   Home Gadgets|   PROD104| STORE03| 15.00|   Florida|    USA|
|               NULL|    Mobile Mart|   PROD105| STORE02| 49.99|     Texas|    USA|
|     Wireless Mouse|Electronics Hub|   PROD106| STORE01|  NULL|California|    USA|
|Mechanical Keyboard|      PC Palace|   PROD107| STORE04|120.00|  New York|    USA|
+-------------------+---------------+----------+--------+------+----------+-------+
only showing top 5 rows



In [25]:
deltaInstance3.history().show(truncate=False)

+-------+-----------------------+------+--------+----------------------+----------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation             |operationParameters                                                                                       |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+-------

In [26]:
spark.sql('update products_table set store_id="STORE03" where product_id="PROD102"')
spark.sql('describe history products_table').show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+--------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                             |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+---

In [27]:
deltaInstance3.update('product_id="PROD104"',set={"store_id":lit("STORE03")})

In [28]:
deltaInstance3.history().show(truncate=False)

+-------+-----------------------+------+--------+----------------------+----------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation             |operationParameters                                                                                       |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+-------

**END OF CODE**